In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
from keras import losses

2024-10-03 21:05:07.680210: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 21:05:07.803365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 21:05:07.849506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 21:05:07.865583: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 21:05:07.947828: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727978709.783590    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978709.881990    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978709.882023    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 128

alphabet = np.array(sorted(set(text)))
sym_to_idx = {}
idx_to_sym = {}

for idx, sym in enumerate(alphabet):
    sym_to_idx[sym] = idx
    idx_to_sym[idx] = sym
    
text_idx = np.array([sym_to_idx[char] for char in text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)

dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727978710.280613    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978710.280656    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978710.280667    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978710.408543    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727978710.408585    1196 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-03

In [6]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), 256),
    keras.layers.LSTM(512, return_sequences=True, stateful=True),
    keras.layers.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=40, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/40
  5/243 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.0978 - loss: 4.5429

2024-10-03 21:05:11.947481: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


243/243 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.2472 - loss: 2.8734
Epoch 2/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.4100 - loss: 2.0458
Epoch 3/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.4691 - loss: 1.8084
Epoch 4/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4994 - loss: 1.6853
Epoch 5/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5197 - loss: 1.6066
Epoch 6/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5349 - loss: 1.5497
Epoch 7/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5470 - loss: 1.5055
Epoch 8/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5568 - loss: 1.4698
Epoch 9/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5650 - loss: 1.4404
Epoch 10/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5720 - loss: 1.4155
Epoch 11/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5780 - loss: 1.3940
Epoch 12/40
243/243 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/ste

In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ''.join(res)
    return generated

In [9]:
print(predict_next(
    sample='б',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=200,
    rnd_power=0.6,
    batch_size=BATCH_SIZE
))

бие нашей, и на всяком деле вы представили закона злодействующих за другой день и возлюбил Иисусова и священников — как вы не для многих делах своих и [сам] не возвращается во Христе Иисусе Христе Богу


In [10]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=sym_to_idx,
    vocabulary=idx_to_sym,
    n_next=100,
    rnd_power=0.2,
    batch_size=BATCH_SIZE
))

1 и совершившись с ними во всём святым деле Божием и поставил вас в сердца ваши и поставил вас в теле
